In [17]:
from datasets import load_dataset
import torchvision.transforms as T
from train_cnn_imagenet import SafeImageNetDataset

train_dataset = load_dataset("imagenet-1k", split="train").shuffle(seed=42)
eval_dataset = load_dataset("imagenet-1k", split="validation").shuffle(seed=42)

mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

# Define the data augmentation and preprocessing pipeline for training images
train_transform = T.Compose([
    T.Lambda(lambda x: x.convert('RGB') if x.mode != 'RGB' else x),  # Ensure 3 channels (convert grayscale to RGB)
    T.RandomResizedCrop(224, scale=(0.08, 1.0)),      # Randomly crop and resize to 224x224 (simulates zoom/scale)
    T.RandomHorizontalFlip(),                         # Randomly flip images horizontally (augmentation)
    T.RandAugment(num_ops=2, magnitude=9),            # Apply 2 random augmentations with magnitude 9 (extra augmentation)
    T.ToTensor(),                                     # Convert PIL Image or numpy.ndarray to tensor and scale to [0, 1]
    T.Normalize(mean, std),                           # Normalize using ImageNet mean and std
    T.RandomErasing(p=0.25, scale=(0.02, 0.1)),       # Randomly erase a rectangle region (extra augmentation, 25% chance)
])

# Define the preprocessing pipeline for evaluation images (no heavy augmentation)
eval_transform = T.Compose([
    T.Lambda(lambda x: x.convert('RGB') if x.mode != 'RGB' else x),  # Ensure 3 channels (convert grayscale to RGB)
    T.Resize(256),                                    # Resize shorter side to 256 pixels
    T.CenterCrop(224),                                # Crop the center 224x224 region
    T.ToTensor(),                                     # Convert to tensor and scale to [0, 1]
    T.Normalize(mean, std),                           # Normalize using ImageNet mean and std
])

def train_transform_fn(examples):
    # Handle both single examples and batches
    if isinstance(examples['image'], list):
        # Batch processing
        examples["pixel_values"] = [train_transform(image) for image in examples["image"]]
    else:
        # Single example processing  
        examples["pixel_values"] = train_transform(examples["image"])

    # Remove the original image to avoid DataLoader issues
    del examples["image"]
    return examples

def eval_transform_fn(examples):
    # Handle both single examples and batches
    if isinstance(examples['image'], list):
        # Batch processing
        examples["pixel_values"] = [eval_transform(image) for image in examples["image"]]
    else:
        # Single example processing
        examples["pixel_values"] = eval_transform(examples["image"])

    # Remove the original image to avoid DataLoader issues
    del examples["image"]
    return examples

# Wrap datasets with safe wrapper to handle EXIF errors on-demand
train_dataset = SafeImageNetDataset(train_dataset, train_transform_fn)
eval_dataset = SafeImageNetDataset(eval_dataset, eval_transform_fn)

Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

In [18]:
train_dataset[0], eval_dataset[0]

({'label': 126,
  'pixel_values': tensor([[[-0.7650, -0.8335, -0.9192,  ..., -2.1179, -2.1179, -2.1179],
           [-0.9363, -0.9534, -1.0048,  ..., -2.1179, -2.1179, -2.1179],
           [-1.1418, -1.1589, -1.1760,  ..., -2.1179, -2.1179, -2.1179],
           ...,
           [-1.8782, -1.8782, -1.8782,  ..., -0.0801, -0.0287,  0.0227],
           [-1.8782, -1.8782, -1.8782,  ..., -0.1486, -0.1486, -0.1486],
           [-1.8782, -1.8782, -1.8782,  ..., -0.1657, -0.2171, -0.2684]],
  
          [[-0.5301, -0.6527, -0.7927,  ..., -2.0357, -2.0357, -2.0357],
           [-0.8102, -0.8452, -0.8978,  ..., -2.0357, -2.0357, -2.0357],
           [-1.0378, -1.0553, -1.0728,  ..., -2.0357, -2.0357, -2.0357],
           ...,
           [-1.7381, -1.7381, -1.7381,  ...,  0.4853,  0.5903,  0.6604],
           [-1.7381, -1.7381, -1.7381,  ...,  0.3978,  0.3627,  0.3452],
           [-1.7381, -1.7381, -1.7381,  ...,  0.3452,  0.2052,  0.1352]],
  
          [[ 0.1476, -0.0964, -0.4101,  ..., -1.8044

In [19]:
train_dataset[0]['pixel_values'].shape, eval_dataset[0]['pixel_values'].shape

(torch.Size([3, 224, 224]), torch.Size([3, 224, 224]))

In [20]:
eval_dataset['label']

[348,
 592,
 376,
 664,
 222,
 760,
 165,
 45,
 987,
 284,
 848,
 641,
 694,
 521,
 3,
 687,
 929,
 760,
 992,
 34,
 82,
 243,
 663,
 744,
 763,
 962,
 149,
 461,
 134,
 235,
 989,
 209,
 486,
 90,
 401,
 443,
 472,
 55,
 387,
 49,
 518,
 408,
 74,
 340,
 331,
 94,
 982,
 485,
 407,
 218,
 766,
 935,
 642,
 157,
 624,
 866,
 513,
 341,
 416,
 955,
 356,
 423,
 657,
 657,
 161,
 568,
 96,
 393,
 953,
 80,
 115,
 559,
 946,
 320,
 217,
 289,
 822,
 192,
 634,
 271,
 245,
 13,
 524,
 943,
 469,
 535,
 923,
 696,
 437,
 703,
 480,
 824,
 647,
 831,
 477,
 950,
 888,
 937,
 892,
 95,
 303,
 411,
 764,
 409,
 36,
 565,
 825,
 108,
 657,
 980,
 27,
 539,
 365,
 958,
 301,
 829,
 385,
 499,
 191,
 538,
 998,
 270,
 402,
 80,
 897,
 55,
 827,
 362,
 619,
 287,
 820,
 700,
 220,
 669,
 392,
 867,
 352,
 499,
 300,
 924,
 471,
 414,
 841,
 728,
 747,
 582,
 987,
 72,
 251,
 803,
 334,
 643,
 752,
 262,
 686,
 695,
 34,
 254,
 457,
 345,
 226,
 656,
 834,
 462,
 973,
 804,
 231,
 197,
 624,
 842,


In [21]:
# Test with actual grayscale image
from PIL import Image
import numpy as np
import torch

print("Testing RGB conversion with actual grayscale image...")

# Create a test grayscale image
gray_array = np.random.randint(0, 256, (300, 400), dtype=np.uint8)
grayscale_image = Image.fromarray(gray_array, mode='L')  # 'L' mode = grayscale

print(f"Original grayscale image mode: {grayscale_image.mode}")
print(f"Original grayscale image size: {grayscale_image.size}")

# Test train transform on grayscale image
print("\nTesting train transform on grayscale image:")
try:
    train_result = train_transform(grayscale_image)
    print(f"✅ Train transform result shape: {train_result.shape}")
    print(f"✅ Successfully converted to {train_result.shape[0]} channels")
    
    # Verify all 3 channels have the same values (since it was grayscale)
    channel_equality = torch.allclose(train_result[0], train_result[1]) and torch.allclose(train_result[1], train_result[2])
    print(f"✅ All 3 channels identical (as expected): {channel_equality}")
    
except Exception as e:
    print(f"❌ Error in train transform: {e}")

# Test eval transform on grayscale image  
print("\nTesting eval transform on grayscale image:")
try:
    eval_result = eval_transform(grayscale_image)
    print(f"✅ Eval transform result shape: {eval_result.shape}")
    print(f"✅ Successfully converted to {eval_result.shape[0]} channels")
    
    # Verify all 3 channels have the same values (since it was grayscale)
    channel_equality = torch.allclose(eval_result[0], eval_result[1]) and torch.allclose(eval_result[1], eval_result[2])
    print(f"✅ All 3 channels identical (as expected): {channel_equality}")
    
except Exception as e:
    print(f"❌ Error in eval transform: {e}")

print("\n🎯 Grayscale to RGB conversion test completed!")


Testing RGB conversion with actual grayscale image...
Original grayscale image mode: L
Original grayscale image size: (400, 300)

Testing train transform on grayscale image:
✅ Train transform result shape: torch.Size([3, 224, 224])
✅ Successfully converted to 3 channels
✅ All 3 channels identical (as expected): False

Testing eval transform on grayscale image:
✅ Eval transform result shape: torch.Size([3, 224, 224])
✅ Successfully converted to 3 channels
✅ All 3 channels identical (as expected): False

🎯 Grayscale to RGB conversion test completed!
